In [ ]:
#Importation des libairies utiles
import numpy as np
from math import*
import matplotlib.pyplot as plt
import random
import scipy
import scipy.stats

plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
class PoissonBandit :
    def __init__(self, means , randomstate = None ):
        ''' Accept an array (means) of K >= 2 positive floats (parameter lambda of each arm)
            and ( optionally ) a seed for a random number generator . '''

        self.means = means
        
        self.random = np.random.RandomState(randomstate)
        
        self.ps_regrets = []

        self.mustar = max(self.means)
        
        self.kstar = np.argmax(means)
        
        self.gaps = means[self.kstar] - means

    def get_K(self):
        ''' Return the number of actions . '''

        K = len(self.means)
        return K
        
    def play(self,action):
        ''' Accept a parameter 0 <= k < K, logs the instant pseudo - regret ,
        and return the realization of Poisson random variable with lambda
        being the mean of the given action. '''

        ps_regrets_inst = self.mustar - self.means[action]

        #méthode de conténation append
        self.ps_regrets.append(self.gaps[action])
         
        # On génère un nombre aléatoire Poisson pour chaque bras
        # ensuit on choisira ensuite le bras action
        
        samples = self.random.poisson(self.means)
        
        #samples contient K nombres (logueur de means) obtenus par échantillonnage sur leur loi
        #de Poisson respective, on choisit le nombre associée à l'action <<action>>
        
        reward = samples[action]
        
        return reward
    
    def get_cumulative_regret(self):
        ''' Return an array of the cumulative sum of pseudo - regret per round . '''
        return np.cumsum(self.ps_regrets)

In [ ]:
#Algorithme Thompson Sampling for Poisson Bandit
#Référence au numéro 3-a) du travail

def ts_poisson(bandit, T, alpha, beta, seed=None, stock_alpha_beta=False):
    '''Play the given bandit over T rounds using the TS strategy for Poisson bandits with given priors 
    alpha (real>0) and beta (real>0), and (optional) random seed 
    stock_alpha_beta is a boolean (true if function have to return alpha and beta of posteriors after each turn)'''
    
    tsrand=np.random.RandomState(seed)
    
    #Obtention du nombre d'actions possibles.
    K = bandit.get_K()  
    
    #Initialisation des distributions
    alpha_actions, beta_actions = np.repeat(alpha,K), np.repeat(beta,K)
    
    ##stockage des alpha beta si demandé
    if(stock_alpha_beta):
        alpha_stock = []
        beta_stock = []
        for k in range(K):
            alpha_stock.append([])
            beta_stock.append([])
            
            alpha_stock[k].append(alpha)
            beta_stock[k].append(beta)
    #####
    
    for t in range(T):  
        
        # Échantillonner selon la loi de gamma, attention la fonction np.random.gamma échantillonne 
        # une valeur selon la loi de gamma avec deuxième paramètre theta=1/beta.
        theta = tsrand.gamma(shape=alpha_actions, scale=1/beta_actions)
        
        # Jouer l'action choisie (l'action avec le theta max)
        k_t = np.argmax(theta)
        
        r_t = bandit.play(k_t)
        
        #On met à jour la distribution posterior de l'action qui a été jouée (k_t).
        
        alpha_actions[k_t] = alpha_actions[k_t] + r_t
        beta_actions[k_t] = beta_actions[k_t] + 1
        
        #On laisse inchangé les distributions posterior des actions non-jouées.
        
        ##stockage des alpha beta si demandé
        if(stock_alpha_beta):
           
            for k in range(K):
                if(k == k_t):
                    alpha_stock[k].append(alpha_actions[k])
                    beta_stock[k].append(beta_actions[k])
                else:
                    alpha_stock[k].append(alpha_stock[k][len(alpha_stock[k])-1])
                    beta_stock[k].append(beta_stock[k][len(beta_stock[k])-1])
        ##############
    
    #Retour des alpha, beta si nécessaire
    if(stock_alpha_beta):
        return(alpha_stock, beta_stock)


In [ ]:
#Code pour tester l'algorithme Thompson Sampling pour des bandits à distributions de Poisson.
#On s'intéresse à un bandit à 2 bras avec moyennes lambda_1=6 et lambda_2=7
#Le code affiche les distributions posteriors à chacun des 100 tours.

banditest=PoissonBandit(np.array([6,7]),10)
alpha, beta = 5/2, 1/2

T=100

al,be = ts_poisson(banditest,T,alpha,beta,seed=9,stock_alpha_beta=True)

x=np.linspace(0,10,100)
for i in range(T+1):
    plt.plot(x,scipy.stats.gamma.pdf(x, al[0][i], scale=1/be[0][i]), color ='red',
             label="posterior de l'action 1")
    plt.plot(x,scipy.stats.gamma.pdf(x, al[1][i], scale=1/be[1][i]), color ='blue',
             label="posterior de l'action 2")
    
    plt.xlabel('Valeurs de $\lambda$ possibles')
    plt.ylabel('Valeurs de la fonction de densité')
    plt.title("Courbes des densités postriors\n pour l'action 1 et l'action 2 après {} tour(s)".format(i))
    
    plt.legend()
    plt.show()


In [ ]:
def graphiques(means, moyennes_priors, variances_priors, T, titre):
    ''' Cette fonction crée le graphique permettant de comparer le pseudo regret cumulatif de l'algorithme Thompson Sampling 
    sur des bras à distribution de Poisson selon les moyennes et variances des distributions gamma des priors. 
    Entrées: 
    les moyennes (means) des différentes instances de PoissonBandit sur lequel les algorithmes font tester.
    les moyennes (moyennes_priors) et variances (variances_priors) sur les priors à tester.
    T: L'horizon sur lequel le test se fait
    title: string représentant le titre du graphique à produite
    '''
    N = means.shape[0] #Donne le nombre d'instances
    n = len(moyennes_priors) #Donne le nombre de tests (priors différents à tester) 
    
    alpha = (moyennes_priors**2)/(variances_priors) #Contient les alpha des différentes priors à tester
    beta = (moyennes_priors)/(variances_priors)     #Contient les beta des différentes priors à tester
    
    cum_regrets=[]
    for i in range(n):
        cum_regrets.append([])
    #cum_regrets contient une liste de n listes prêtes à contenir les regrets cumulatifs pour chacun des tests.
    
    colors = ['red', 'blue', 'green', 'yellow', 'purple', 'brown', 'black']
    
    for i in range(n):
        
        #i donne le numéro de la prior à utiliser
        
        for k in range(N):
            
            banditest = PoissonBandit(means[k,],randomstate=k)
            ts_poisson(banditest, T, alpha=alpha[i], beta=beta[i], seed=k)
            cum_regrets[i].append(banditest.get_cumulative_regret())
            
        cum_regrets_moy = np.apply_along_axis(np.mean, 0, cum_regrets[i])
        cum_regrets_sd = np.apply_along_axis(np.std, 0, cum_regrets[i])
        
        plt.plot(range(T), cum_regrets_moy, color=colors[i],
        label = r'$\alpha_0$ = {}, $\beta_0$ = {},  $\mu_0$ = {}, $\sigma^2_0$ = {}'.format(alpha[i],beta[i], moyennes[i],variances[i]))
        plt.plot(range(T), cum_regrets_moy + cum_regrets_sd, color = colors[i], linestyle='dashed')
    
    plt.xlabel('Pas de temps')
    plt.ylabel('Pseudo regret cumulatif moyen')
    plt.title(titre)
    lgd=plt.legend(bbox_to_anchor=(1.05, 1))
        
    ##Ne pas décommenter, c'est seulement pour gérer le sauvegarde des figures dans un fichier
    #plt.savefig("variance_trop_petites_N=200.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.savefig("variance_trop_grande_N=200.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.savefig("moyenne_decentree_variance_bonne_N=200.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.savefig("moyenne_decentree_variance_petite_N=200.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# On génère ici N instances de bandits à deux bras à distribution de Poissons avec les moyennes choisient 
# uniformément dans l'intervalle[0,10]

generation=np.random.RandomState(10)

N, T = 200, 1000

means = 10*generation.rand(N,2)

#Expérience sur des variances trop petites

#on teste des priors différents (ici la moyenne est toujours de 5 (centre de l'intervalle))
#teste de l'effet d'une variance trop petite

moyennes = np.array([5,5,5,5,5])
variances = np.array([10,5,1,1/2,1/4])

titre='Test pour Thompson Sampling, N={}, T={},\n impact de la variance trop petite des priors'.format(N,T)

graphiques(means, moyennes, variances, T, titre)


In [ ]:
# On génère ici N instances de bandits à deux bras à distribution de Poissons avec les moyennes choisient 
# uniformément dans l'intervalle[0,10]

generation=np.random.RandomState(10)

N, T = 200, 1000

means = 10*generation.rand(N,2)

#Expérience sur des variances trop grandes

#on teste des priors différents (ici la moyenne est toujours de 5 (centre de l'intervalle))
#test de l'effet d'une variance trop grande 

moyennes = np.array([5,5,5,5,5])
variances = np.array([10,20,50,100,1000])

titre='Test pour Thompson Sampling, N={}, T={},\n impact de la variance trop grande des priors'.format(N,T)

graphiques(means, moyennes, variances, T, titre)

In [ ]:
# On génère ici N instances de bandits à deux bras à distribution de Poissons avec les moyennes choisient 
# uniformément dans l'intervalle[10,20]

generation=np.random.RandomState(10)

N, T = 200, 1000

#Moyennes lambda générées entre 10 et 20.
means = 10*generation.rand(N,2)+10

#Effet de la moyenne de la loi prior avec variance «bonne»

#variance égale à la longueur de l'intervalle (10)
#on teste des priors de moyennes différentes, moyennes = 5,12,15,17,25

moyennes = np.array([5,12,15,17,25])
variances = np.array([10,10,10,10,10])

titre='Test pour Thompson Sampling, N={}, T={},\n impact de la moyenne des priors dans le cas avec une <<bonne variance>> des priors'.format(N,T)

graphiques(means, moyennes, variances, T, titre)

In [ ]:
# On génère ici N instances de bandits à deux bras à distribution de Poissons avec les moyennes choisient 
# uniformément dans l'intervalle[10,20]

generation=np.random.RandomState(10)

N, T = 200, 1000

#Moyennes lambda générées entre 10 et 20.
means = 10*generation.rand(N,2)+10

#Effet de la moyenne de la loi prior avec variance plus petite

#variance des priors égalent à 3
#on teste des priors différents, moyennes = 9,12,15,17,21

moyennes = np.array([9,12,15,17,21])
variances = np.array([3,3,3,3,3])

titre='Test pour Thompson Sampling, N={}, T={},\n impact de la moyenne des priors dans le cas avec une variance petite des priors'.format(N,T)

graphiques(means, moyennes, variances, T, titre)